## Baroclinic instability

### 2 level model

In [1]:
anim_dir = "animations/";

In [2]:
using Pkg
# pkg"add Oceananigans, JLD2, Plots"

In [3]:
using Oceananigans
using Oceananigans.Units
using Oceananigans.Models.HydrostaticFreeSurfaceModels: ImplicitFreeSurface
using Oceananigans.OutputReaders: FieldTimeSeries
using Plots, Printf

In [5]:
grid = RectilinearGrid(size = (128, 2),
                       x = (0, 1000kilometers), z = (-400meters, 0),
                       topology = (Periodic, Flat, Bounded))

128×1×2 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CPU with 1×0×1 halo
├── Periodic x ∈ [0.0, 1.0e6)  regularly spaced with Δx=7812.5
├── Flat y
└── Bounded  z ∈ [-400.0, 0.0] regularly spaced with Δz=200.0

In [6]:
coriolis = BetaPlane(f₀=1e-4, β=1.5e-11)

BetaPlane{Float64}(f₀=0.0001, β=1.5e-11)

In [10]:
background_parameters = ( α = 10 * coriolis.f₀, # s⁻¹, geostrophic shear
                          f = coriolis.f₀,      # s⁻¹, Coriolis parameter
                          N = 1e-3,            # s⁻¹, buoyancy frequency
                         Lz = grid.Lz)         # m, ocean depth

(α = 0.001, f = 0.0001, N = 0.001, Lz = 400.0)

In [11]:
using Oceananigans.Fields: BackgroundField

# Background fields are defined via functions of x, y, z, t, and optional parameters
U(x, y, z, t, p) = + p.α * (z + p.Lz)
B(x, y, z, t, p) = - p.α * p.f * y + p.N^2 * z

U_field = BackgroundField(U, parameters=background_parameters)
B_field = BackgroundField(B, parameters=background_parameters)

BackgroundField{typeof(B), NamedTuple{(:α, :f, :N, :Lz), NTuple{4, Float64}}}
├── func: B (generic function with 1 method)
└── parameters: (α = 0.001, f = 0.0001, N = 0.001, Lz = 400.0)

In [16]:
drag_coefficient = 1e-4

@inline drag_u(x, y, t, u, v, cᴰ) = - cᴰ * u * sqrt(u^2 + v^2)
@inline drag_v(x, y, t, u, v, cᴰ) = - cᴰ * v * sqrt(u^2 + v^2)

drag_bc_u = FluxBoundaryCondition(drag_u, field_dependencies=(:u, :v), parameters=drag_coefficient)
drag_bc_v = FluxBoundaryCondition(drag_v, field_dependencies=(:u, :v), parameters=drag_coefficient)

u_bcs = UVelocityBoundaryConditions(grid, bottom = drag_bc_u)
v_bcs = VVelocityBoundaryConditions(grid, bottom = drag_bc_v)

LoadError: UndefVarError: UVelocityBoundaryConditions not defined

In [20]:
Pkg.status("Oceananigans")

      Status `~/Documents/GitHub/geophysviz/Project.toml`
  [9e8cae18] Oceananigans
